# Flame Regime GAN - Quick Demo
## Generate Oscillatory Flame Behavior in Minutes

This notebook demonstrates the key features of the Flame GAN model in just a few cells.

In [ ]:
# 1. Install required packages
!pip install torch matplotlib seaborn pandas numpy scikit-learn tqdm scipy
print("Dependencies installed!")

In [ ]:
# 2. Load the GAN model
# Upload flame_gan_colab.py to Colab first!
exec(open('flame_gan_colab.py').read())
print("Flame GAN model loaded!")

In [ ]:
# 3. Create and train the GAN
import matplotlib.pyplot as plt
import numpy as np

print("Creating Flame GAN...")
gan = FlameGAN()

print("Generating synthetic oscillatory training data...")
sequences, conditions = gan.create_oscillatory_data(n_samples=100)
print(f"Generated {len(sequences)} training sequences")

print("Training GAN (this takes a few minutes)...")
gan.train(sequences, conditions, epochs=100, batch_size=16)
print("Training completed!")

In [ ]:
# 4. Generate sequences for different flame conditions
test_conditions = [
    (0.8, 0.2, "Stable - Low φ, Low u"),
    (0.9, 0.35, "Stable - Medium φ, Medium u"),
    (1.0, 0.5, "Transition - High φ, Medium u"),
    (1.2, 0.7, "Unstable - High φ, High u")
]

fig, axes = plt.subplots(len(test_conditions), 3, figsize=(18, 12))

for i, (phi, u, description) in enumerate(test_conditions):
    # Generate sequence
    sequence = gan.generate_sequence(phi, u)
    
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
    components = ['X1', 'X2', 'X3']
    
    for j in range(3):
        axes[i, j].plot(sequence[:, j], color=colors[j], linewidth=2.5, alpha=0.8)
        axes[i, j].set_title(f'{description} - {components[j]}', fontweight='bold')
        axes[i, j].set_xlabel('Time Step')
        axes[i, j].set_ylabel('Value')
        axes[i, j].grid(True, alpha=0.3)
        
        # Add statistics
        mean_val = np.mean(sequence[:, j])
        std_val = np.std(sequence[:, j])
        axes[i, j].text(0.02, 0.98, f'μ={mean_val:.3f}\nσ={std_val:.3f}', 
                       transform=axes[i, j].transAxes, verticalalignment='top',
                       bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.suptitle('🔥 Generated Oscillatory Flame Sequences', y=1.01, fontsize=18, fontweight='bold')
plt.show()

print("✅ Generated sequences for different flame regimes!")

In [ ]:
# 5. Phase space analysis
stable_seq = gan.generate_sequence(0.8, 0.3)
unstable_seq = gan.generate_sequence(1.2, 0.6)

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# X1 vs X2 phase space
axes[0].scatter(stable_seq[:, 0], stable_seq[:, 1], alpha=0.6, s=40, label='Stable (φ=0.8, u=0.3)', color='blue')
axes[0].scatter(unstable_seq[:, 0], unstable_seq[:, 1], alpha=0.6, s=40, label='Unstable (φ=1.2, u=0.6)', color='red')
axes[0].set_xlabel('X1')
axes[0].set_ylabel('X2')
axes[0].set_title('Phase Space: X1 vs X2')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# X1 vs X3 phase space
axes[1].scatter(stable_seq[:, 0], stable_seq[:, 2], alpha=0.6, s=40, label='Stable', color='blue')
axes[1].scatter(unstable_seq[:, 0], unstable_seq[:, 2], alpha=0.6, s=40, label='Unstable', color='red')
axes[1].set_xlabel('X1')
axes[1].set_ylabel('X3')
axes[1].set_title('Phase Space: X1 vs X3')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# X2 vs X3 phase space
axes[2].scatter(stable_seq[:, 1], stable_seq[:, 2], alpha=0.6, s=40, label='Stable', color='blue')
axes[2].scatter(unstable_seq[:, 1], unstable_seq[:, 2], alpha=0.6, s=40, label='Unstable', color='red')
axes[2].set_xlabel('X2')
axes[2].set_ylabel('X3')
axes[2].set_title('Phase Space: X2 vs X3')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('🔬 Phase Space Analysis', y=1.02, fontsize=16, fontweight='bold')
plt.show()

print("✅ Phase space analysis completed!")

In [ ]:
# 6. Frequency analysis
from scipy.fft import fft, fftfreq

stable_seq = gan.generate_sequence(0.8, 0.3)
unstable_seq = gan.generate_sequence(1.2, 0.6)

fig, axes = plt.subplots(2, 3, figsize=(18, 10))

sequences = [stable_seq, unstable_seq]
labels = ['Stable (φ=0.8, u=0.3)', 'Unstable (φ=1.2, u=0.6)']
components = ['X1', 'X2', 'X3']
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']

for i, (seq, label) in enumerate(zip(sequences, labels)):
    for j, (comp, color) in enumerate(zip(components, colors)):
        # Compute FFT
        n = len(seq[:, j])
        fft_vals = fft(seq[:, j])
        freqs = fftfreq(n, 1.0)
        
        # Plot positive frequencies only
        pos_mask = freqs > 0
        freqs_pos = freqs[pos_mask]
        fft_pos = np.abs(fft_vals[pos_mask])
        
        axes[i, j].semilogy(freqs_pos[:15], fft_pos[:15], color=color, linewidth=2)
        axes[i, j].set_title(f'{label} - {comp} Frequency Spectrum')
        axes[i, j].set_xlabel('Frequency')
        axes[i, j].set_ylabel('Magnitude (log scale)')
        axes[i, j].grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('📊 Frequency Analysis of Generated Sequences', y=1.02, fontsize=16)
plt.show()

print("✅ Frequency analysis completed!")

In [ ]:
# 7. Save model and generate dataset
print("💾 Saving trained model...")
gan.save_model('/content/flame_gan_demo_model.pth')

# Generate a small dataset
print("📋 Generating sample dataset...")
phi_values = [0.8, 1.0, 1.2]
u_values = [0.3, 0.5, 0.7]

dataset_summary = []
for phi in phi_values:
    for u in u_values:
        sequence = gan.generate_sequence(phi, u)
        filename = f"Demo_Phi_{phi:.1f}_u_{u:.1f}_generated.txt".replace('.', 'p')
        
        # Save sequence file
        np.savetxt(f'/content/{filename}', sequence, fmt='%.6E', delimiter='\t')
        
        # Collect summary info
        regime = 'stable' if phi < 0.9 or (phi >= 1.0 and u < 0.3) else 'unstable'
        dataset_summary.append({
            'filename': filename,
            'phi': phi,
            'u': u,
            'regime': regime,
            'x1_mean': np.mean(sequence[:, 0]),
            'x1_std': np.std(sequence[:, 0]),
            'oscillation_amplitude': np.std(sequence[:, 0]) * 2  # Rough oscillation measure
        })

# Save summary
import pandas as pd
summary_df = pd.DataFrame(dataset_summary)
summary_df.to_csv('/content/demo_dataset_summary.csv', index=False)

print("✅ Files saved:")
print("  📄 flame_gan_demo_model.pth - Trained model")
print("  📊 demo_dataset_summary.csv - Dataset summary")
print(f"  🗂️ {len(dataset_summary)} sequence files")

# Display summary
print("\n📊 Generated Dataset Summary:")
print(summary_df)

## Demo Complete

You've successfully:
1. Installed dependencies
2. Loaded the Flame GAN model
3. Trained on synthetic oscillatory data
4. Generated sequences for different flame regimes
5. Analyzed phase space behavior
6. Performed frequency analysis
7. Saved the model and generated dataset

### Next Steps:
- **Use Your Own Data**: Load real flame measurement data for training
- **Parameter Studies**: Explore different φ and u ranges
- **Extended Training**: Train for more epochs for better results
- **Advanced Analysis**: Implement additional signal processing techniques

### Files Generated:
- `flame_gan_demo_model.pth` - Your trained model
- `demo_dataset_summary.csv` - Summary of generated sequences
- Multiple `.txt` files with generated flame sequences

The model is now ready to generate oscillatory flame behavior for any φ and u values!